In [14]:
import os
import random
import openai
import json
from Keys import openai_keys
import re
openai.organization = openai_keys['organization']
openai.api_key = openai_keys['api_key']

In [15]:
# Load the JSON data from the file
with open('C:/Users\Admin\Favorites\Links\Everything\Stuff\PostMetacritic\Source/reddit-oppenheimer-286.json', 'r') as json_file:
    comments_by_post = json.load(json_file)


In [9]:
class Post:
    def __init__(self, title, ratings, raw):
        self.title = title
        self.ratings = ratings
        self.comments = raw

In [13]:
submissions = list()
i = 0
total = 8
breakingBad = False
for title, comments in comments_by_post.items():
    if i >= total:
        break
    coms = list()
    rates = list()
    for c in comments:
        coms.append(c)
        inp = input(str(i) + "/" + total + " " + str(title) + ":\n"+ str(c))
        if inp.lower() == 'stop':
            breakingBad = True
            break
        rate = int(inp)
        rates.append(rate if 0 <= rate <= 2 else 1)
    if breakingBad:
        break
    submissions.append(Post(title, rates, coms))
    i += 1


ValueError: invalid literal for int() with base 10: '0stop'

In [ ]:
# Step 3: Serialize the list to JSON
json_data = json.dumps([obj.__dict__ for obj in submissions], indent=4)

# Save the JSON data to a file
with open("Datasets/HumanGenerated/dataset" + str(random.randint(1, 9999)), "w") as json_file:
    json_file.write(json_data)

In [ ]:
redditSentiment = list()
iter = 0
for title, comments in comments_by_post.items():
    prompts = [{"role": "system", "content":
        'Analyse the sentiment of 16 Reddit comments under a post - "' + str(title) + '", use the delimiter "\n\n==COMMENT==\n\n" to distinguish individual comments. Your goal is to write sentiment analysis for each comment towards the topic - "movie Oppenheimer", in a format RATING - KEYWORDS. Where RATING is a number from 0 to 10, with 0 representing drastically negative attitude to the topic, 10 for a drastically positive, or 5 for neutral or a comment unrelated to a topic. KEYWORDS are 2-3 words or phrases taken unchanged from a comment, which are most indicative of reasons for such attitude, separate them with a comma (,). Use a semicolon (;) as the delimiter between each comment analysis. Example: "8 - Exciting visuals, Nolan, direction; 4 - Confusing plot, slow pacing; 5 - long duration, too many characters;"'}]
    delimeter = "\n\n==COMMENT==\n\n"
    prompt = delimeter.join(comments)
    prompts.append({"role": "user", "content": prompt})
    chat = openai.ChatCompletion.create(model="gpt-3.5-turbo-16k", messages=prompts)
    reply = chat.choices[0].message.content
    redditSentiment.append(Post(title, reply.split(';'), prompt))
    iter += 1
    print(str(iter) + "/" + str(len(comments_by_post)))